In [4]:
import warnings

warnings.filterwarnings("ignore")  # ignore warnings

import os
import re

from time import time
from tqdm import tqdm  # to track the processing of loop
import pandas as pd  # data preprocessing
import numpy as np  # numerical python processing(linear algebra)
import matplotlib.pyplot as plt  # visualization
import seaborn as sns  # Visualization on top of Matplotlib

# metrics to assess the model performance
from sklearn.metrics import f1_score, precision_score, recall_score


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve 

# For Deep Learning models
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pickle  # to save model
from pathlib import Path

In [5]:
dataset_dir = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Dataset//'

In [6]:
import os
import pickle
from gensim.models import Word2Vec
import pandas as pd


df= pd.read_csv(dataset_dir + 'AllCombined.csv')

print(df)


         Unnamed: 0                                               func  target
0                 0  ((rte_pktmbuf_mtod_offset(mbuf,\tstruct tcp_hd...       1
1                 1  void CWE121_Stack_Based_Buffer_Overflow__CWE13...       1
2                 2  BER_LOG_FN ber_int_log_proc = NULL;\n\nFILE *b...       1
3                 3  do { \\nint rest = sizeof(buf) - 1 - strlen(bu...       1
4                 4  void\n_bfd_pei_swap_sym_in (bfd * abfd, void *...       1
...             ...                                                ...     ...
1609425     1609425                  b'visitICmpInst(ICmpInst &I) { \n       0
1609426     1609426  b'path_node_delete_chain(path_node * head)\n {...       0
1609427     1609427  b'addrconf6_start (NMDevice *self)\n { \n\tNMD...       0
1609428     1609428  b'Next(const SQObjectPtr &refpos, SQObjectPtr ...       0
1609429     1609429  b'EffectHealMaxHealth(SpellEffectIndex )\n { \...       0

[1609430 rows x 3 columns]


In [4]:
df.head()

,Unnamed: 0,func,target
0,0,"((rte_pktmbuf_mtod_offset(mbuf,\tstruct tcp_hd...",1
1,1,void CWE121_Stack_Based_Buffer_Overflow__CWE13...,1
2,2,BER_LOG_FN ber_int_log_proc = NULL;\n\nFILE *b...,1
3,3,do { \\nint rest = sizeof(buf) - 1 - strlen(bu...,1
4,4,"void\n_bfd_pei_swap_sym_in (bfd * abfd, void *...",1


In [7]:
import code_tokenize
def my_tokenizer(text):
    text = re.sub(r'\b\d+\.\d+\b', lambda x: str(float(x.group())), text)
    tokens = code_tokenize.tokenize(text, lang="c", syntax_error="ignore")
    tokens_list = [str(x) for x in tokens]
    return tokens_list

In [8]:
# tokenize the data that can be used by embeddings
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer_path = "F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Tokenizers//tokenizerClubbed.pkl"
retrain = False

if os.path.exists(tokenizer_path) and not retrain:
    print("[INFO] Loading saved tokenizer")
    with open(tokenizer_path, "rb") as tokenizer_file:
        tokenizer = pickle.load(tokenizer_file)
else:
    t0 = time()
    print("[INFO] Fitting tokenizer on TRAIN data...")
    tokenizer = Tokenizer(lower=False, analyzer=my_tokenizer)  # used in the research paper
    tokenizer.fit_on_texts(X_train.apply(lambda x: str(x)))
    # Save the tokenizer for future use
    with open(tokenizer_path, "wb") as tokenizer_file:
        pickle.dump(tokenizer, tokenizer_file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Time taken to fit and save the tokenizer: {time() - t0} s")

[INFO] Loading saved tokenizer


In [9]:
X_train = [str(code) for code in df.func]
X_train_tokens = tokenizer.texts_to_sequences(X_train)

# 2. Create an empty list to store the sentence strings
sentences = []

# 3. Iterate through the tokenized sequences and create sentence strings
for token_sequence in X_train_tokens:
  sentence = [tokenizer.index_word[token] for token in token_sequence]
  sentences.append(sentence)


# Now, the 'sentences' variable will contain the token sequences as strings

In [10]:
for te in sentences[:5]:
    print("NICE\n")
    print('_______________________________________________________')
    print(te, "\n")  # unpacking the list and using newline as separator

NICE

_______________________________________________________
['(', '(', 'rte_pktmbuf_mtod_offset', '(', 'mbuf', ',', 'struct', 'tcp_hdr', '*', ',', 'mbuf', '->', 'l3_len', '+', 'mbuf', '->', 'l2_len', ')', '->', 'data_off', ')', '>>', '4', ')', '', 'enum', 'ethtool_stringset', '{', 'ETH_SS_TEST', '=', '0', ',', 'ETH_SS_STATS', ',', '}', ';', 'struct', 'ena_stats', '{', 'char', 'name', '[', 'ETH_GSTRING_LEN', ']', ';', 'int', 'stat_offset', ';', '}', ';', '.', 'name', '=', '.', 'stat_offset', '=', 'offsetof', '(', 'struct', 'ena_stats_', '}', 'ENA_STAT_ENTRY', '(', 'stat', ',', 'rx', ')', 'ENA_STAT_ENTRY', '(', 'stat', ',', 'tx', ')', 'ENA_STAT_ENTRY', '(', 'stat', ',', 'dev', ')', 'uint32_t', 'ena_alloc_cnt', ';', 'static', 'const', 'struct', 'ena_stats', 'ena_stats_global_strings', '[', ']', '=', '{', 'ENA_STAT_GLOBAL_ENTRY', '(', 'wd_expired', ')', ',', 'ENA_STAT_GLOBAL_ENTRY', '(', 'dev_start', ')', ',', 'ENA_STAT_GLOBAL_ENTRY', '(', 'dev_stop', ')', ',', '}', ';', 'static', 'const

In [11]:
import os
import pickle
from gensim.models import Word2Vec


output_dir = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined'

embedding_dims = [50]  # Different embedding sizes
window_size = 10
min_count = 1

# Iterate over different embedding sizes
for embedding_dim in embedding_dims:
    
    word2vec_model = Word2Vec(sentences, vector_size=embedding_dim, window=window_size, min_count=min_count)

    # Save the Word2Vec model using pickle with a specific name
    model_name = f'word2vec_C_{embedding_dim}_model.pkl'
    model_path = os.path.join(output_dir, model_name)
    with open(model_path, 'wb') as f:
        pickle.dump(word2vec_model, f)

    print(f"Embedding model (size {embedding_dim}) saved at:", model_path)


Embedding model (size 50) saved at: F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined\word2vec_C_50_model.pkl


In [12]:
from gensim.models import FastText

output_dir = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined'
embedding_dims = [50]  # Different embedding sizes

for embedding_dim in embedding_dims:
    
    fasttext_model = FastText(sentences, vector_size=embedding_dim, window=window_size, min_count=min_count)

    # Save the FastText model using pickle with a specific name
    model_name = f'fasttext_C_{embedding_dim}_model.pkl'
    model_path = os.path.join(output_dir, model_name)
    with open(model_path, 'wb') as f:
        pickle.dump(fasttext_model, f)

    print(f"Embedding model (size {embedding_dim}) saved at:", model_path)


Embedding model (size 50) saved at: F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined\fasttext_C_50_model.pkl


In [13]:
import pickle

vocab_path = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined//word2vec_c_50_model.pkl'
# Load the Word2Vec model from the pickle file
with open(vocab_path, 'rb') as f:
    word2vec_model = pickle.load(f)

# Get the vocabulary size
vocab_size = len(word2vec_model.wv.key_to_index)

# Get a few vocabulary words
few_vocab_words = list(word2vec_model.wv.key_to_index.keys())[:100]  # Adjust the number as needed

# Get embeddings for some words
word_embeddings = {}
words_to_check = ['kmknkn', 'if', 'then']  # Add words you want to check
for word in words_to_check:
    if word in word2vec_model.wv.key_to_index:
        word_embeddings[word] = word2vec_model.wv.get_vector(word)

# Print information
print("Vocabulary size:", vocab_size)
print("Few vocabulary words:", few_vocab_words)
print("Embeddings for some words:", word_embeddings)


Vocabulary size: 2398616
Few vocabulary words: [' ', 'e', 't', 'r', '_', 'i', '\\n', 's', 'n', 'a', '(', ')', '\\t', ';', 'o', 'c', 'l', 'd', ',', 'p', 'u', 'f', '=', 'm', 'g', 'b', '-', '>', 'h', ' \\n', '*', 'E', '"', '{', 'L', '}', 'v', '0', 'T', 'S', 'N', 'x', 'y', 'R', 'A', 'I', 'k', 'C', '.', 'O', '&', 'w', '->', '1', 'P', 'D', 'U', "'", ':', 'G', 'M', '[', ']', 'F', '+', '2', 'if', '!', 'B', '<', 'z', 'j', 'q', '', 'V', 'H', '%', '3', '|', 'K', 'return', 'X', '\\\\', 'W', '4', '==', 'Y', 'int', '8', '6', '5', 'NULL', 'struct', '/', '!=', 'else', '&&', 'case', 'char', 'Q']
Embeddings for some words: {'if': array([ -1.1720202 ,  -2.8918865 ,   1.5284976 ,  -2.393013  ,
        -2.7506855 ,  -4.9240694 ,  -4.5635805 ,  -4.8384295 ,
       -10.0903635 ,  -3.0323153 ,  -4.633209  ,   2.333248  ,
         7.8795633 ,  -5.711137  ,   8.341126  ,   3.2002711 ,
         9.228747  ,   1.4424143 ,  -0.47015756,  -3.183608  ,
         4.6659565 ,  -0.4264905 ,  -2.9381716 ,  -4.873555  ,
  

In [14]:
import pickle

# Load the Word2Vec model from the pickle file
vocab_path = 'F://Development_//Deep Learning//Basic Dataset//BTP//1//Outputs//Embedding Models Combined//fasttext_c_50_model.pkl'
with open(vocab_path, 'rb') as f:
    word2vec_model = pickle.load(f)

# Get the vocabulary size
vocab_size = len(word2vec_model.wv.key_to_index)

# Get a few vocabulary words
few_vocab_words = list(word2vec_model.wv.key_to_index.keys())[:100]  # Adjust the number as needed

# Get embeddings for some words
word_embeddings = {}
words_to_check = ['word1', 'if', 'then']  # Add words you want to check
for word in words_to_check:
    if word in word2vec_model.wv.key_to_index:
        word_embeddings[word] = word2vec_model.wv.get_vector(word)

# Print information
print("Vocabulary size:", vocab_size)
print("Few vocabulary words:", few_vocab_words)
print("Embeddings for some words:", word_embeddings)


Vocabulary size: 2398616
Few vocabulary words: [' ', 'e', 't', 'r', '_', 'i', '\\n', 's', 'n', 'a', '(', ')', '\\t', ';', 'o', 'c', 'l', 'd', ',', 'p', 'u', 'f', '=', 'm', 'g', 'b', '-', '>', 'h', ' \\n', '*', 'E', '"', '{', 'L', '}', 'v', '0', 'T', 'S', 'N', 'x', 'y', 'R', 'A', 'I', 'k', 'C', '.', 'O', '&', 'w', '->', '1', 'P', 'D', 'U', "'", ':', 'G', 'M', '[', ']', 'F', '+', '2', 'if', '!', 'B', '<', 'z', 'j', 'q', '', 'V', 'H', '%', '3', '|', 'K', 'return', 'X', '\\\\', 'W', '4', '==', 'Y', 'int', '8', '6', '5', 'NULL', 'struct', '/', '!=', 'else', '&&', 'case', 'char', 'Q']
Embeddings for some words: {'word1': array([ 0.25417322,  2.7727377 ,  0.19617556,  1.2230752 ,  1.9860455 ,
       -0.8090251 , -2.4425304 ,  0.7488693 ,  2.251443  ,  0.71544176,
        1.6895881 ,  3.4277618 ,  3.6619227 , -0.94520026, -0.32190916,
       -2.498619  ,  2.5836523 ,  2.823853  ,  2.541669  , -0.86662173,
        2.9498248 , -0.7955227 , -0.95712465,  1.6373038 , -2.206827  ,
       -0.9178747